In [ ]:
# !pip install transformers deepspeed-mii --upgrade

In [2]:
from transformers import pipeline, AutoTokenizer
import torch
import time
import deepspeed

pipe = pipeline("text-generation",
                model="bigscience/bloom-560m",
                device=0,
                torch_dtype=torch.float16)

In [3]:
pipe.model = deepspeed.init_inference(
    pipe.model,
    mp_size=1,
    dtype=torch.half,
    replace_method='auto',
    replace_with_kernel_inject=True
)

[2022-11-17 08:12:43,701] [INFO] [logging.py:68:log_dist] [Rank -1] DeepSpeed info: version=0.7.5, git-hash=unknown, git-branch=unknown
[2022-11-17 08:12:43,702] [INFO] [logging.py:68:log_dist] [Rank -1] quantize_bits = 8 mlp_extra_grouping = False, quantize_groups = 1
Using /home/ec2-user/.cache/torch_extensions/py38_cu111 as PyTorch extensions root...
Detected CUDA files, patching ldflags
Emitting ninja build file /home/ec2-user/.cache/torch_extensions/py38_cu111/transformer_inference/build.ninja...
Building extension module transformer_inference...
Allowing ninja to set a default number of workers... (overridable by setting the environment variable MAX_JOBS=N)
ninja: no work to do.
Loading extension module transformer_inference...
Time to load transformer_inference op: 0.3389394283294678 seconds
[2022-11-17 08:12:44,857] [INFO] [logging.py:68:log_dist] [Rank -1] DeepSpeed-Inference config: {'layer_id': 0, 'hidden_size': 1024, 'intermediate_size': 4096, 'heads': 16, 'num_hidden_layer

In [4]:
tokenizer = AutoTokenizer.from_pretrained("bigscience/bloom-560m")
start_text = "Testing BLOOM-560M without DeepSpeed"
tokens_start_text = len(tokenizer(start_text, return_tensors="pt").input_ids[0])

In [5]:
new_tokens = 500

t0 = time.time()
result = pipe(start_text, max_new_tokens=new_tokens)
t1 = time.time()

------------------------------------------------------
Free memory : 11.855957 (GigaBytes)  
Total memory: 14.560913 (GigaBytes)  
Requested memory: 0.093750 (GigaBytes) 
Setting maximum total tokens (input + output) to 1024 
------------------------------------------------------


In [6]:
gen_text = result[0]['generated_text']
tokens_gen_text = len(tokenizer(gen_text, return_tensors="pt").input_ids[0])
total_new_tokens_generated = tokens_gen_text - tokens_start_text
throughput = (total_new_tokens_generated) / (t1 - t0)

In [7]:
print(f"""Tokens generated: {total_new_tokens_generated}
Time: {t1 - t0:.1f} seconds
Tokens per second: {throughput:.1f}
Latency: {1000 / throughput:.1f} ms""")

Tokens generated: 500
Time: 7.4 seconds
Tokens per second: 67.9
Latency: 14.7 ms
